In [1]:
import re
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

to be able to see all 2000 rows (the maximum number of rows in our data) together:

In [201]:
# pd.set_option('display.max_rows', 2000)

loading data into 4 dataframes:

In [2]:
books = pd.read_csv('books.csv')
checkouts = pd.read_csv('checkouts.csv')
customers = pd.read_csv('customers.csv')
libraries = pd.read_csv('libraries.csv')

First we skim over each table to get a sense of the data type, anomalies, NaNs, etc. :

In [3]:
libraries

,id,name,street_address,city,region,postal_code
0,226-222@5xc-kc4-fpv,Multnomah County Library Capitol Hill,10723 SW capitol Hwy,Portland,OR,97219
1,23v-222@5xc-jv7-v4v,Multnomah County Library Northwest,2300 NW Thurman St,NaN,or,NaN
2,222-222@5xc-jvf-skf,Multnomah County Library St Johns,7510 N Charleston Ave,portland,or,97203
3,227-222@5xc-jww-btv,Multnomah County Library Hillsdale,1525 SW Sunset blvd,Portland,or,-97239
4,22d-222@5xc-kcy-8sq,Multnomah County Library Sellwood Moreland,7860 SE 13th AVE,Portland,OR,97202
5,223-222@5xc-jxr-tgk,MULTNOMAH County Library Woodstock,6008 se 49TH AVE,Portland,OR,-97206
6,zzw-224@5xc-jwv-2rk,Multnomah County Library Central,801 SW 10th Ave,Portland,NaN,97205
7,zzw-223@5xc-jv7-ct9,Friends OF the multnomah COUNTY Library,522 SW 5th Ave,NaN,OR,97204
8,226-222@5xc-jxj-7yv,Multnomah County Library Belmont,1038 SE CESAR E CHAVEZ blvd,Portland,OR,97214
9,zzw-222@5xc-knn-c5z,Multnomah County Library Holgate,7905 SE Holgate Blvd,Portland,OR,NaN


The "libraries" table contains id, name, street_address, city, region, and the postal code. All libraries are located in Portland, OR so there is no information in columns city and region for us. There might be differences in book-returning behavior based on the overall (zip code) or exact (address) location. So we keep them, but we need to do some cleaning and transforming. The zip codes contain some extraneous characters around them (like "_", "-" or "#") and one NaN. We remove the extra characters and we find the right zip codes for the NaN values by googling the corresponding addresses (in this case there only 3 so we can do that):

In [4]:
libraries.postal_code[1] = '97210' #'2300 NW Thurman St'
libraries.postal_code[9] = '97206' #'7905 SE Holgate Blvd'
libraries.postal_code[12]= '97212' #'216 ne Knott st'

def get_num(x):
    return re.sub('[^0-9]', '', x)

libraries.postal_code = libraries.postal_code.apply(get_num)

Then we transform the addresses to latitude-longitudes to have a rough estimate of neighbourhood:

In [5]:
from geopy.geocoders import Nominatim
def get_lat_long(address):
    geolocator = Nominatim()
    full_adr = address + ', Portland, OR'
    location = geolocator.geocode(full_adr)
    return location.latitude, location.longitude

lat_long = libraries.street_address.apply(get_lat_long)

libraries['latitude'] = lat_long.map(lambda x:x[0])
libraries['longitude'] = lat_long.map(lambda x:x[1])
libraries

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


,id,name,street_address,city,region,postal_code,latitude,longitude
0,226-222@5xc-kc4-fpv,Multnomah County Library Capitol Hill,10723 SW capitol Hwy,Portland,OR,97219,45.448003,-122.725449
1,23v-222@5xc-jv7-v4v,Multnomah County Library Northwest,2300 NW Thurman St,NaN,or,97210,45.535336,-122.699145
2,222-222@5xc-jvf-skf,Multnomah County Library St Johns,7510 N Charleston Ave,portland,or,97203,45.590036,-122.751053
3,227-222@5xc-jww-btv,Multnomah County Library Hillsdale,1525 SW Sunset blvd,Portland,or,97239,45.479834,-122.694087
4,22d-222@5xc-kcy-8sq,Multnomah County Library Sellwood Moreland,7860 SE 13th AVE,Portland,OR,97202,45.467632,-122.652746
5,223-222@5xc-jxr-tgk,MULTNOMAH County Library Woodstock,6008 se 49TH AVE,Portland,OR,97206,45.479005,-122.612107
6,zzw-224@5xc-jwv-2rk,Multnomah County Library Central,801 SW 10th Ave,Portland,NaN,97205,45.519201,-122.683202
7,zzw-223@5xc-jv7-ct9,Friends OF the multnomah COUNTY Library,522 SW 5th Ave,NaN,OR,97204,45.519585,-122.676793
8,226-222@5xc-jxj-7yv,Multnomah County Library Belmont,1038 SE CESAR E CHAVEZ blvd,Portland,OR,97214,45.515267,-122.622676
9,zzw-222@5xc-knn-c5z,Multnomah County Library Holgate,7905 SE Holgate Blvd,Portland,OR,97206,45.490566,-122.582281


The number of pages for books are in string format. Let's make them integer: 

In [6]:
books.pages = books.pages.apply(get_num)
books.pages = books.pages.apply(int)

For date of publishing, we transform it to the number of days from the first available date in the list (1830-01-01), i.e. the oldest book in the library:

In [7]:
min_date = min(books.publishedDate.apply(pd.to_datetime))
books.publishedDate = [x.days for x in books.publishedDate.apply(pd.to_datetime)-min_date]

Defining a transformer to transform the lists in "categories" and "authors" columns to dictionaries like {"author1": 1, "author2": 1} so we can then feed it into sklearns's DictVectorizer for one-hot encoding:

In [8]:
class DictEncoder(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        l=[]
        for xx in X:
            for x in xx:
                if pd.isnull(x):
                    l.append({'':0})
                else:
                    d = {x.replace("']","").replace("['",""):1}
                    l.append(d)

        return pd.Series(l)

Slightly different DictEncoder to handle the authors:

In [9]:
class DictEncoder2(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        l=[]
        for x in X:
            if not isinstance(x,str):
                l.append({'':0})
            else:
                names = x.replace("'","").replace("[","").replace("]","").split(",")
                d = {a:1 for a in names}
                l.append(d)

        return pd.Series(l)

For decoding Publisher and other single-strings (not like authors and categories which are lists) we can use just OneHotEncoder.

For Title, we use CountVectorizer:

In [57]:
countvec = CountVectorizer()

Price is also in string format. Let's make it in float type:

In [11]:
def get_num_point(x):
    if isinstance(x, str):
        return re.sub('[^0-9\.0-9]', '', x)
    else:
        return np.nan
books.price = books.price.apply(get_num_point)
books.price = books.price.apply(float)

In [12]:
customers

,id,name,street_address,city,state,zipcode,birth_date,gender,education,occupation
0,df83ec2d0d409395c0d8c2690cfa8b67,Cynthia Barnfield,44 NE Meikle Pl,Portland,Oregon,97213.0,2009-09-10,female,High School,NaN
1,6aec7ab2ea0d67161dac39e5dcabd857,Elizabeth Smith,7511 SE Harrison St,Portland,Oregon,97215.0,1956-12-15,female,College,Blue Collar
2,0c54340672f510fdb9d2f30595c1ab53,Richard Pabla,1404 SE Pine St,Portland,Oregon,97214.0,1960-12-18,male,College,Education & Health
3,f0d9ce833ddc1f73c1e0b55bdebf012e,Charles Baker,12271 N Westshore Dr,Portland,Oregon,97217.0,2105-07-19,male,Graduate Degree,SALES
4,3720379163f6b46944db6c98c0485bfd,Ronald Lydon,5321 NE Skyport Way,NaN,Oregon,97218.0,1961-03-14,male,Graduate Degree,Blue Collar
...,...,...,...,...,...,...,...,...,...,...
1995,ae55f0b71b8b8e91945cd9a91b6e45ee,JOE Roberts,7331 NE Killingsworth St,Portland,NaN,97218.0,1955-05-23,male,Others,Business & Finance
1996,07fe407cc889ea21a8bdc04c305960b1,Matthew Coniglio,1908 NW Harborside Dr,Vancouver,washington,98660.0,1975-11-10,male,Others,Business & Finance
1997,9a2194fcd4f0f326f0ca334450e16a93,Earl Grier,22 NE graham ST,Portland,OREGON,97212.0,2007-10-02,male,Others,Education & Health
1998,01a598a05c48fdd18461d6411f51a109,Rogelio Richmann,7000 NE Airport Way,Portland,OREGON,97218.0,2001-02-19,male,College,Business & Finance


Next is the customers table. We can transform customers' addresses into latitude-longtitue to get an idea of how distant they are from the library. It might be some information to use there:

In [ ]:
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

def do_geocode(address):
    geopy = Nominatim()
    try:
        return geopy.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

def get_lat_long(addresses):
    res = []
    for adr in addresses:
        geolocator = Nominatim()
        if isinstance(adr, str):
            location = do_geocode(adr)
            if location:
                res.append((location.latitude, location.longitude))
            else:
                res.append(np.nan)
        else:
            res.append(np.nan)
    return pd.Series(res)

customers['full_address'] = customers['street_address'] + ', ' + customers['city'] + ', ' + customers['state']
lat_long = customers.full_address.apply(get_lat_long)

customers['latitude'] = lat_long.map(lambda x:x[0])
customers['longitude'] = lat_long.map(lambda x:x[1])

I had to do it in a separete notbook and load here to save time:

In [13]:
df = pd.read_csv('customers_lat_long.csv')
customers['latitude'] = df.Latitude
customers['longitude'] = df.Longitude

Next, let's take a look at customers' birthdates. There are many wierd dates here:

In [14]:
customers.birth_date = customers.birth_date.apply(pd.to_datetime)
min(customers.birth_date)

Timestamp('1800-02-19 00:00:00')

In [15]:
max(customers.birth_date)

Timestamp('2120-08-25 00:00:00')

  92 customers are born before 1880:

In [16]:
sum(customers.birth_date.apply(pd.to_datetime)<pd.to_datetime('1900'))

92

And 94 are born in the future:

In [17]:
sum(customers.birth_date.apply(pd.to_datetime)>pd.to_datetime('2023'))

94

I have been trying to find out a common pattern(s) to guess a possible typo in the data entering, so I can guess the correct year based on that. But no pattern stands out clearly. So for now, I'm going to replace these (94+92) birthdates with NaN so later in the pipeline we impute them with other NaNs. 

In [18]:
customers[customers.birth_date.apply(pd.to_datetime)>pd.to_datetime('2023')]

,id,name,street_address,city,state,zipcode,birth_date,gender,education,occupation,latitude,longitude
3,f0d9ce833ddc1f73c1e0b55bdebf012e,Charles Baker,12271 N Westshore Dr,Portland,Oregon,97217.0,2105-07-19,male,Graduate Degree,SALES,45.613485,-122.692632
10,a407f60337af8e554a1f85f99092a4e3,Peter Watkins,1847 SW Englewood Dr,Lake Oswego,Oregon,97034.0,2102-12-05,male,Graduate Degree,Blue Collar,45.436048,-122.696375
15,59f403406fe6523aba62e51117050f8d,John Milkins,16907 SW 72nd Ave,Portland,Oregon,97224.0,2108-11-02,male,High School,Others,45.397998,-122.749109
34,31fe87a21a2a7c39f4a03d0b736ab2a0,Glen Winslow,12728 SE Mill St,Portland,Oregon,97233.0,2106-08-04,male,High School,Tech,45.509741,-122.532434
53,f9372de3c8ea501601aa3fb59ec0f524,James Klein,1720 N Shaver St,Portland,Oregon,97227.0,2119-02-10,male,Graduate DEGREE,Sales,45.551979,-122.685827
...,...,...,...,...,...,...,...,...,...,...,...,...
1877,e9eb7de60a48d036ac6040540f6b3727,Evelyn Siragusa,17850 Stafford Rd,West Linn,Oregon,97068.0,2108-09-19,female,High School,Admin & Support,45.396412,-122.688356
1884,507daa4146217ea10e1e56a279563272,Helen Kozel,3907 NE 15th Ave,Portland,Oregon,_97212.0,2118-03-14,female,HIGH school,Education & Health,45.550926,-122.650623
1912,b3340b15223925b5e701b64afd0861c4,David Taylor,4364 SW Dickinson St,Portland,Oregon,97219.0,2111-10-05,male,College,Blue Collar,45.447371,-122.722155
1950,af15da3e14b19e4380b128972c8deff9,Tresa Villiard,1999 SE Water Ave,Portland,Oregon,97214.0,2101-05-23,female,College,Tech,45.510389,-122.665922


Let's replace these values with NaN, so we can later properly impute them:

In [19]:
customers.loc[customers.birth_date<pd.to_datetime('1880'), 'birth_date'] = np.nan
customers.loc[customers.birth_date>pd.to_datetime('2023'), 'birth_date'] = np.nan

Next let's look at gender. We will use OneHotEncoder to encode gender (as well as education and occupation). But before that let's take a look to see our raw data make sense:

In [20]:
customers.gender.unique()

array(['female', ' female', 'male', 'MALE', nan, 'male ', 'FEMALE',
       ' male', 'female '], dtype=object)

As you can see, instead of having three unique valuses ('male', 'female', and NaN), we have duplicates of both 'male' and 'female' values due to slight differences in typing, e.g. s space entered at the beginning, or uppercase/lowercase differences. Let's fix this:

In [21]:
customers['gen2'] = pd.Series([isinstance(x, str) and x.lower().strip() for x in customers.gender])
customers.loc[customers['gen2']==False,'gen2'] = np.nan

In [22]:
customers['gen2'].value_counts()

male      967
female    932
Name: gen2, dtype: int64

The same is hapening with education. Lot's of variations in typing of the same values:

In [23]:
customers.education.unique()

array(['High School', 'College', 'Graduate Degree', 'Others',
       'High  School', ' Others', 'Others ', 'Graduate DEGREE', nan,
       'College ', 'Graduate  Degree', 'High school', ' College',
       'OTHERS', 'High  School ', 'High   School', 'others',
       'high School', 'college', 'graduate DEGREE', 'Graduate degree',
       'Graduate  Degree ', 'High School  ', 'GRADUATE degree', 'COLLEGE',
       'High School ', 'HIGH school', 'GRADUATE Degree',
       ' Graduate  Degree', 'Graduate   Degree', 'HIGH School',
       'High SCHOOL', 'HIGH SCHOOL', 'GRADUATE DEGREE', ' High School ',
       'graduate Degree', ' High  School', 'high SCHOOL',
       ' Graduate Degree ', 'Graduate Degree '], dtype=object)

We can fix it by:

In [24]:
for i, edu in enumerate(customers.education):
    if isinstance(edu,str):
        if 'school' in edu.lower():
            customers.loc[i, 'education'] = 'highschool'
        elif 'college' in edu.lower():
            customers.loc[i, 'education'] = 'college'
        elif 'graduate' in edu.lower():
            customers.loc[i, 'education'] = 'graduate'
        elif 'other' in edu.lower():
            customers.loc[i, 'education'] = 'other'

Same with occupation:

In [25]:
customers.occupation.unique()

array([nan, 'Blue Collar', 'Education & Health', 'SALES', 'Tech', 'Sales',
       'Others', 'Business & Finance', ' Business &  Finance ',
       'Education  & Health', 'EDUCATION & Health', ' Others',
       'Admin & Support', 'Tech ', 'blue Collar', 'Blue   Collar',
       'Education &    Health', 'Admin & SUPPORT',
       'Education  &  Health ', 'Business & FINANCE', 'Blue  Collar',
       'sales', 'Sales ', 'BUSINESS & Finance', 'Business  &  Finance ',
       'others', 'Business &  Finance', ' Blue  Collar',
       'education & Health', ' Blue Collar', 'Blue collar', 'BLUE collar',
       'Admin &  Support', 'BLUE COLLAR', ' Blue Collar ', 'OTHERS',
       'tech', 'Education  &  Health', ' Sales', 'Others ',
       'Education & HEALTH', 'Education  & Health ', 'ADMIN & Support',
       'Admin &   Support', 'Business  & Finance  ',
       'Education   &  Health', 'BLUE Collar', 'Education &  Health',
       'Blue COLLAR', 'Blue  Collar ', ' Education  & Health',
       ' Admin & S

In [26]:
for i, ocu in enumerate(customers.occupation):
    if isinstance(ocu,str):
        if 'education' in ocu.lower():
            customers.loc[i, 'occupation'] = 'education_health'
        elif 'sale' in ocu.lower():
            customers.loc[i, 'occupation'] = 'sales'
        elif 'tech' in ocu.lower():
            customers.loc[i, 'occupation'] = 'tech'
        elif 'other' in ocu.lower():
            customers.loc[i, 'occupation'] = 'other'
        elif 'business' in ocu.lower():
            customers.loc[i, 'occupation'] = 'business_finance'
        elif 'admin' in ocu.lower():
            customers.loc[i, 'occupation'] = 'admin_support'
        elif 'blue' in ocu.lower():
            customers.loc[i, 'occupation'] = 'bluecollar'

Similar to the publishing date of books, I transformed the birthdate of customers to the distance from the oldest one:

In [27]:
customers.birth_date = customers.birth_date - min(customers.birth_date)
for i in range(len(customers)):
    if customers.loc[i,'birth_date']:
        customers.loc[i,'birth_date'] = customers.loc[i,'birth_date'].days
    else:
        customers.loc[i,'birth_date'] = np.nan

Finally, we reach to the last table to clean-up: checkouts

In [28]:
checkouts

,id,patron_id,library_id,date_checkout,date_returned
0,-xFj0vTLbRIC,b071c9c68228a2b1d00e6f53677e16da,225-222@5xc-jtz-hkf,2019-01-28,2018-11-13
1,HUX-y4oXl04C,8d3f63e1deed89d7ba1bf6a4eb101373,223-222@5xc-jxr-tgk,2018-05-29,2018-06-12
2,TQpFnkku2poC,4ae202f8de762591734705e0079d76df,228-222@5xc-jtz-hwk,2018-11-23,2019-01-24
3,OQ6sDwAAQBAJ,f9372de3c8ea501601aa3fb59ec0f524,23v-222@5xc-jv7-v4v,2018-01-15,2018-04-25
4,7T9-BAAAQBAJ,2cf3cc3b9e9f6c608767da8d350f77c9,225-222@5xc-jtz-hkf,2018-12-31,1804-01-23
...,...,...,...,...,...
1995,rNbuDwAAQBAJ,91871955f3641857832766ac3f5a0b95,222-222@5xc-jv5-nt9,2018-07-19,2018-08-12
1996,rcrCAgAAQBAJ,ad08956eb20efb746af650f906d439cf,22d-222@5xc-kcy-8sq,2018-03-07,2018-03-13
1997,F44fAQAAMAAJ,026262cc3454149303074c4113b5f118,226-222@5xc-jxj-7yv,2018-06-17,2018-06-27
1998,Ci1HAQAAMAAJ,08b29865e58e9b2aabff9684a703acf0,223-222@5xc-jxr-tgk,2018-12-10,2018-12-29


Most of the dates are in 'yyyy-mm-dd' format. But some of them are not:

In [29]:
for i in range(len(checkouts)):
    date1 = checkouts.date_checkout[i]
    date2 = checkouts.date_returned[i]
    if isinstance(date1, str):
        if not re.match('\d{4}-\d{2}-\d{2}', date1):
            print(date1)
    if isinstance(date2, str):
        if not re.match('\d{4}-\d{2}-\d{2}', date2):
            print('                 ', date2)
    

                  2018|10|15%
                  2018/08/03
2018/12/31 
                   20180612
 20180629
%2018 06 04
2018 03 31%
%2018/11/24
 20180319
                  20181111
 2018 01 03
                  2018/02/17
                   2018|05|13
%2018/02/26
                  2018 11 29%
20180804 
                   20181006
                  20180927
%2018 07 23
                  20180120
                   2018|11|15
2018/07/07%
 20181121
                  %2018 04 27
                  2018|07|06%
2018|03|26
2018/01/11
                   20190109
                  2018|04|07%
20180913
2018/11/29
                  20190109
                  20180407%
                  %20180729
                   2018|07|13
                  2018|11|12 
                  %2018 07 22
 2018|06|28
                  2018/07/12
2018 08 04
20180108%
                  2018|04|25
20180126
                  2018/04/13%
                  2018|08|15
20181125 
                  2018 10 06
2018 10 17
       

One way to get all these format right is to ignore any non-numeric characters (we got '20010302') and then parse the result based on yyyymmdd:

In [30]:
for i in range(len(checkouts)):
    date1 = checkouts.date_checkout[i]
    date2 = checkouts.date_returned[i]
    if isinstance(date1, str):
        date1 = re.sub('\D', '', date1)
        date1 = date1[:4]+'-'+date1[4:6]+'-'+date1[6:]
        checkouts.date_checkout[i] = pd.to_datetime(date1)
    if isinstance(date2, str):
        date2 = re.sub('\D', '', date2)
        date2 = date2[:4]+'-'+date2[4:6]+'-'+date2[6:]
        checkouts.date_returned[i] = pd.to_datetime(date2)

After getting the format right, again observe some very far off dates in both checkout_dates and returned_dates:

In [31]:
sum(checkouts.date_checkout>pd.to_datetime('2023'))

84

In [32]:
sum(checkouts.date_returned>pd.to_datetime('2023'))

95

In [33]:
sum(checkouts.date_checkout<pd.to_datetime('1900'))

90

In [34]:
sum(checkouts.date_returned<pd.to_datetime('1900'))

82

Since any obvious pattern of typos show up here, we simply replace these dates with NaN:

In [35]:
checkouts.loc[checkouts.date_checkout>pd.to_datetime('2023'),'date_checkout']=np.nan
checkouts.loc[checkouts.date_checkout<pd.to_datetime('1900'),'date_checkout']=np.nan
checkouts.loc[checkouts.date_returned>pd.to_datetime('2023'),'date_returned']=np.nan
checkouts.loc[checkouts.date_returned<pd.to_datetime('1900'),'date_returned']=np.nan

We define a new column 'delay' which is date_returned - date_checkout. For negative cases we are going to replace them with NaN:

In [36]:
checkouts['delay'] = checkouts.date_returned - checkouts.date_checkout

We go thorugh the new column 'delay' and transform it to binary values: 1 if it is delayed more than 28 days, 0 if it is less than 28 days. NaNs for negative values of delays:

In [37]:
for i in range(len(checkouts)):
    if checkouts.delay[i].days<0:
        checkouts.loc[i,'delay'] = np.nan
    elif checkouts.delay[i].days>28:
        checkouts.loc[i,'delay'] = 1
    elif checkouts.delay[i].days<=28:
        checkouts.loc[i,'delay'] = 0

We have 1451 valid data points:

In [38]:
sum([1 for x in checkouts.delay if x==0 or x==1])

1451

Lets make a clean X and y arrays for our machine learning. We need to match the ids so the right X's from libraries, books, and customers table match with the right y (delay) from checkouts table. For this I make separete Xs for each of the main players: person, book, library -->

In [39]:
valid_indices = [x==0 or x==1 for x in checkouts.delay]
Y = checkouts[valid_indices][['id', 'patron_id', 'library_id', 'delay']]
Y

,id,patron_id,library_id,delay
1,HUX-y4oXl04C,8d3f63e1deed89d7ba1bf6a4eb101373,223-222@5xc-jxr-tgk,0
2,TQpFnkku2poC,4ae202f8de762591734705e0079d76df,228-222@5xc-jtz-hwk,1
3,OQ6sDwAAQBAJ,f9372de3c8ea501601aa3fb59ec0f524,23v-222@5xc-jv7-v4v,1
6,CW-7tHAaVR0C,dd9f34e9d65126a2b02003d8ac60aaa4,22c-222@5xc-jwj-pvz,0
8,t1e3BWziAc8C,3b85b2c7b424618f533329018e9a11d5,222-222@5xc-jv5-nt9,0
...,...,...,...,...
1995,rNbuDwAAQBAJ,91871955f3641857832766ac3f5a0b95,222-222@5xc-jv5-nt9,0
1996,rcrCAgAAQBAJ,ad08956eb20efb746af650f906d439cf,22d-222@5xc-kcy-8sq,0
1997,F44fAQAAMAAJ,026262cc3454149303074c4113b5f118,226-222@5xc-jxj-7yv,0
1998,Ci1HAQAAMAAJ,08b29865e58e9b2aabff9684a703acf0,223-222@5xc-jxr-tgk,0


In [40]:
X_library = libraries[['id', 'latitude', 'longitude']]
X_library

,id,latitude,longitude
0,226-222@5xc-kc4-fpv,45.448003,-122.725449
1,23v-222@5xc-jv7-v4v,45.535336,-122.699145
2,222-222@5xc-jvf-skf,45.590036,-122.751053
3,227-222@5xc-jww-btv,45.479834,-122.694087
4,22d-222@5xc-kcy-8sq,45.467632,-122.652746
5,223-222@5xc-jxr-tgk,45.479005,-122.612107
6,zzw-224@5xc-jwv-2rk,45.519201,-122.683202
7,zzw-223@5xc-jv7-ct9,45.519585,-122.676793
8,226-222@5xc-jxj-7yv,45.515267,-122.622676
9,zzw-222@5xc-knn-c5z,45.490566,-122.582281


In [41]:
X_merge_Y = pd.merge(
    Y,
    X_library,
    how="inner",
    left_on='library_id',
    right_on='id'
)
X_merge_Y.rename(columns={'patron_id':'customer_id',
                          'id_x': 'book_id',
                          'latitude': 'lat_library',
                          'longitude':'lng_library'}, inplace = True)
X_merge_Y = X_merge_Y[['customer_id', 'book_id', 'library_id', 'lat_library', 'lng_library', 'delay']]
X_merge_Y

,customer_id,book_id,library_id,lat_library,lng_library,delay
0,8d3f63e1deed89d7ba1bf6a4eb101373,HUX-y4oXl04C,223-222@5xc-jxr-tgk,45.479005,-122.612107,0
1,7fd53f66a6d4cd12875487af4008bb08,MOewp6bCFfgC,223-222@5xc-jxr-tgk,45.479005,-122.612107,1
2,ceb72a116d42d8d441e234f80ecf8bb9,iAzrAAAAIAAJ,223-222@5xc-jxr-tgk,45.479005,-122.612107,0
3,7a1f47edd69d69da4c5f95a6a95f75ee,ygoFAAAAQAAJ,223-222@5xc-jxr-tgk,45.479005,-122.612107,0
4,55ed00061e3e9092b8d390ff157b1166,eBcFQ3DUcQEC,223-222@5xc-jxr-tgk,45.479005,-122.612107,0
...,...,...,...,...,...,...
1446,719b0ad56ea4b9b4360e174be1b3b1e1,Ox4DAAAAQAAJ,222-222@5xc-jvf-skf,45.590036,-122.751053,0
1447,e29ba2bf18c9d091421af6f0c8e8a7f7,MOewp6bCFfgC,222-222@5xc-jvf-skf,45.590036,-122.751053,0
1448,4520c746cf75798c777d0f0a1bacb62c,xSYmAQAAIAAJ,222-222@5xc-jvf-skf,45.590036,-122.751053,0
1449,71325ff160af04c414dd03b58774f23a,-xFj0vTLbRIC,222-222@5xc-jvf-skf,45.590036,-122.751053,0


Adding book informations:

In [42]:
books.rename(columns={'id': 'book_id'}, inplace=True)
books

,book_id,title,authors,publisher,publishedDate,categories,price,pages
0,hVFwAAAAQBAJ,Ogilvy on Advertising,['David Ogilvy'],Vintage,67093.0,['Social Science'],72.99,320
1,bRY9AAAAYAAJ,Foreign Publications for Advertising American ...,['United States. Bureau of Foreign and Domesti...,NaN,30315.0,['Advertising'],469.99,654
2,ZapAAAAAIAAJ,Advertising and the Public Interest,"['John A. Howard', 'James Hulbert']",NaN,52230.0,['Advertising'],372.00,784
3,A-HthMfF5moC,Profitable Advertising,NaN,NaN,23376.0,['Advertising'],240.99,559
4,4Z9JAAAAMAAJ,Report of the Federal Trade Commission on Dist...,['United States. Federal Trade Commission'],NaN,41637.0,['Government publications'],539.00,757
...,...,...,...,...,...,...,...,...
235,W58mAQAAIAAJ,Political and Commercial Control of the Minera...,['United States. Dept. of the Interior'],NaN,32141.0,NaN,153.00,503
236,frzDCQAAQBAJ,Water Resources Management IV,"['C.A. Brebbia', 'A. Kungolos']",WIT Press,64775.0,['Nature'],563.50,780
237,mQTxAAAAMAAJ,"Department Publications - State of California,...",['California. Dept. of Water Resources'],NaN,60265.0,['Hydrology'],216.50,748
238,lMkmAQAAMAAJ,Technical Report - South Carolina Marine Resou...,['South Carolina. Marine Resources Division'],NaN,54421.0,['Marine resources'],11.50,236


In [43]:
X_merge_Y = pd.merge(
    X_merge_Y,
    books,
    how="inner",
    on='book_id'
)
X_merge_Y

,customer_id,book_id,library_id,lat_library,lng_library,delay,title,authors,publisher,publishedDate,categories,price,pages
0,8d3f63e1deed89d7ba1bf6a4eb101373,HUX-y4oXl04C,223-222@5xc-jxr-tgk,45.479005,-122.612107,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752
1,77a595f9fb214a8877c49f0c1e354f7e,HUX-y4oXl04C,23v-222@5xc-jv7-v4v,45.535336,-122.699145,1,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752
2,3fa0ad163470ae588dbff044328de641,HUX-y4oXl04C,222-222@5xc-jv5-nt9,45.562459,-122.671537,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752
3,230ea2e764085948ddf9d27371698b3a,HUX-y4oXl04C,zzw-222@5xc-knn-c5z,45.490566,-122.582281,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752
4,8380f7f05a183e9b17f133d5e8a78707,HUX-y4oXl04C,226-222@5xc-jxj-7yv,45.515267,-122.622676,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,acd2c173eef280129c8782c164907fa9,N9V34rWEOGcC,222-222@5xc-kkw-bzf,45.516730,-122.538471,1,Financial Management,['Khan & Jain'],Tata McGraw-Hill Education,64768.0,['Corporations'],181.99,743
1447,3db968ed889159f4abb1b112a8221b4c,C4lNAAAAYAAJ,22d-222@5xc-kcy-8sq,45.467632,-122.652746,0,Mechanics of Engineering ...,['Irving Porter Church'],NaN,24837.0,['Engineering'],267.50,449
1448,abcbf151a1dc88305fe989666d8ff52e,P1AoDwAAQBAJ,zzw-222@5xc-jv4-789,45.549385,-122.651389,0,Reality In Advertising,['Rosser Reeves'],Lulu.com,68460.0,['Business & Economics'],384.00,506
1449,f106e9d626ec8ef9c1384efa24ef4de5,P1AoDwAAQBAJ,222-222@5xc-jvf-skf,45.590036,-122.751053,0,Reality In Advertising,['Rosser Reeves'],Lulu.com,68460.0,['Business & Economics'],384.00,506


Now let's merge the final table (customers):

In [44]:
customers.rename(columns={'id': 'customer_id',
                          'latitude': 'lat_customer',
                          'longitude': 'lng_customer'}, inplace=True)
X_customers = customers[['customer_id', 'birth_date', 'gender', 'education', 'occupation', 'lat_customer', 'lng_customer']]
X_customers

,customer_id,birth_date,gender,education,occupation,lat_customer,lng_customer
0,df83ec2d0d409395c0d8c2690cfa8b67,21800,female,highschool,NaN,45.523487,-122.618735
1,6aec7ab2ea0d67161dac39e5dcabd857,2538,female,college,bluecollar,45.509127,-122.586057
2,0c54340672f510fdb9d2f30595c1ab53,4002,male,college,education_health,45.520622,-122.651425
3,f0d9ce833ddc1f73c1e0b55bdebf012e,NaN,male,graduate,sales,45.613485,-122.692632
4,3720379163f6b46944db6c98c0485bfd,4088,male,graduate,bluecollar,NaN,NaN
...,...,...,...,...,...,...,...
1995,ae55f0b71b8b8e91945cd9a91b6e45ee,1966,male,other,business_finance,NaN,NaN
1996,07fe407cc889ea21a8bdc04c305960b1,9442,male,other,business_finance,45.634387,-122.700972
1997,9a2194fcd4f0f326f0ca334450e16a93,21091,male,other,education_health,45.542684,-122.665952
1998,01a598a05c48fdd18461d6411f51a109,18675,male,college,business_finance,45.589159,-122.593494


In [45]:
X_merge_Y = pd.merge(
    X_merge_Y,
    X_customers,
    how="inner",
    on='customer_id'
)
X_merge_Y

,customer_id,book_id,library_id,lat_library,lng_library,delay,title,authors,publisher,publishedDate,categories,price,pages,birth_date,gender,education,occupation,lat_customer,lng_customer
0,8d3f63e1deed89d7ba1bf6a4eb101373,HUX-y4oXl04C,223-222@5xc-jxr-tgk,45.479005,-122.612107,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,5500,female,graduate,tech,45.486876,-122.637210
1,77a595f9fb214a8877c49f0c1e354f7e,HUX-y4oXl04C,23v-222@5xc-jv7-v4v,45.535336,-122.699145,1,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,11634,female,other,sales,45.485904,-122.704089
2,3fa0ad163470ae588dbff044328de641,HUX-y4oXl04C,222-222@5xc-jv5-nt9,45.562459,-122.671537,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,10019,female,NaN,sales,45.568862,-122.619258
3,230ea2e764085948ddf9d27371698b3a,HUX-y4oXl04C,zzw-222@5xc-knn-c5z,45.490566,-122.582281,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,5399,male,NaN,education_health,45.454092,-122.586715
4,8380f7f05a183e9b17f133d5e8a78707,HUX-y4oXl04C,226-222@5xc-jxj-7yv,45.515267,-122.622676,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,14886,female,graduate,tech,45.552633,-122.591968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,acd2c173eef280129c8782c164907fa9,N9V34rWEOGcC,222-222@5xc-kkw-bzf,45.516730,-122.538471,1,Financial Management,['Khan & Jain'],Tata McGraw-Hill Education,64768.0,['Corporations'],181.99,743,6416,male,graduate,education_health,45.503783,-122.479257
1447,3db968ed889159f4abb1b112a8221b4c,C4lNAAAAYAAJ,22d-222@5xc-kcy-8sq,45.467632,-122.652746,0,Mechanics of Engineering ...,['Irving Porter Church'],NaN,24837.0,['Engineering'],267.50,449,10226,NaN,highschool,business_finance,45.491433,-122.677191
1448,abcbf151a1dc88305fe989666d8ff52e,P1AoDwAAQBAJ,zzw-222@5xc-jv4-789,45.549385,-122.651389,0,Reality In Advertising,['Rosser Reeves'],Lulu.com,68460.0,['Business & Economics'],384.00,506,8103,male,graduate,admin_support,45.541832,-122.622660
1449,f106e9d626ec8ef9c1384efa24ef4de5,P1AoDwAAQBAJ,222-222@5xc-jvf-skf,45.590036,-122.751053,0,Reality In Advertising,['Rosser Reeves'],Lulu.com,68460.0,['Business & Economics'],384.00,506,21809,female,college,education_health,45.616595,-122.720332


Now we are ready to fit some models. But before that, we need to split our data into training anf test sets:

In [46]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(X_merge_Y, test_size=.2)

In [47]:
df_train.shape, df_test.shape

((1160, 19), (291, 19))

Let's start with a logistic model on all the features we got:

In [73]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer

y = df_train.delay.values.reshape(-1,1).astype('int')

numerical_features = ColumnTransformer([('num', 'passthrough', ['lat_library', 'lng_library', 
                                                                'lat_customer',	'lng_customer',
                                                                'price','pages',
                                                                'publishedDate', 'birth_date'])])

categorical_features= ColumnTransformer([('onehot', 'passthrough', ['publisher', 
                                                                    'categories', 'gender',
                                                                    'education', 'occupation'])])

dictvec1_feature = ColumnTransformer([('dictvec1', 'passthrough', ['categories'])])

dictvec2_feature = ColumnTransformer([('dictvec2', 'passthrough', ['authors'])])

countvec_feature = ColumnTransformer([('countvec', 'passthrough', ['title'])])

pipe_num = Pipeline([('num', numerical_features),
                     ('impute', SimpleImputer()),
                     ('scale', StandardScaler())])

pipe_onehot = Pipeline([('cat', categorical_features),
                        ('onehot', OneHotEncoder())])

pipe_dictvec1 = Pipeline([('dict1', dictvec1_feature),
                          ('dictenc1', DictEncoder()),
                          ('dictvec', DictVectorizer())
                          ])

pipe_dictvec2 = Pipeline([('dict2', dictvec2_feature),
                          ('dictenc2', DictEncoder2()),
                          ('dictvec', DictVectorizer())
                          ])

pipe_nlp = Pipeline([('title', countvec_feature),
                     ('flatten', FunctionTransformer(lambda x: x.reshape(-1,))),
                     ('countvec', countvec)])

union = FeatureUnion([('pipe_num', pipe_num), 
                      ('pipe_onehot',pipe_onehot),
                      ('pipe_dictvec1', pipe_dictvec1),
                      ('pipe_dictvec2', pipe_dictvec2),
                      ('pipe_nlp', pipe_nlp)])

union.fit(X_merge_Y)
X_tr = union.transform(df_train)

In [122]:
union.get_params()

{'n_jobs': None, 'transformer_list': [('pipe_num', Pipeline(steps=[('num',
                    ColumnTransformer(transformers=[('num', 'passthrough',
                                                     ['lat_library', 'lng_library',
                                                      'lat_customer',
                                                      'lng_customer', 'price',
                                                      'pages', 'publishedDate',
                                                      'birth_date'])])),
                   ('impute', SimpleImputer()), ('scale', StandardScaler())])),
  ('pipe_onehot', Pipeline(steps=[('cat',
                    ColumnTransformer(transformers=[('onehot', 'passthrough',
                                                     ['publisher', 'categories',
                                                      'gender', 'education',
                                                      'occupation'])])),
                   ('onehot', One

Let's fit a simple logistic model:

In [74]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model.fit(X_tr, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [78]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
y_test = df_test.delay.values.reshape(-1,1).astype('int')
x_test = union.transform(df_test)
y_pred = log_model.predict(x_test)

print('accuracy: ', accuracy_score(y_test, y_pred))

accuracy:  0.8969072164948454


Let's try a random forest:

In [79]:
from sklearn.ensemble import RandomForestClassifier 
clf = RandomForestClassifier()
clf.fit(X_tr,y)

y_pred = clf.predict(x_test)
print('accuracy: ', accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


accuracy:  0.8969072164948454


GradientBoosting:

In [80]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf.fit(X_tr,y)

y_pred = clf.predict(x_test)
print('accuracy: ', accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy:  0.9175257731958762


Gaussian

In [86]:
from sklearn.gaussian_process import GaussianProcessClassifier
clf = GaussianProcessClassifier()
clf.fit(X_tr.toarray(),y)

y_pred = clf.predict(x_test.toarray())
print('accuracy: ', accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy:  0.8900343642611683


KNeighbours:

In [87]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf.fit(X_tr.toarray(),y)

y_pred = clf.predict(x_test.toarray())
print('accuracy: ', accuracy_score(y_test, y_pred))

accuracy:  0.9106529209621993


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Support Vector Machines

In [88]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_tr.toarray(),y)

y_pred = clf.predict(x_test.toarray())
print('accuracy: ', accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy:  0.9140893470790378


In [93]:
from sklearn.svm import SVC
clf = SVC(C=5)
clf.fit(X_tr.toarray(),y)

y_pred = clf.predict(x_test.toarray())
print('accuracy: ', accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy:  0.9140893470790378


Naive Bayes

In [94]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_tr.toarray(),y)

y_pred = clf.predict(x_test.toarray())
print('accuracy: ', accuracy_score(y_test, y_pred))

accuracy:  0.6391752577319587


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [104]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB(var_smoothing=1)
clf.fit(X_tr.toarray(),y)

y_pred = clf.predict(x_test.toarray())
print('accuracy: ', accuracy_score(y_test, y_pred))

accuracy:  0.9140893470790378


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


more ensemble methods: Votingclassifier

In [109]:
from sklearn.ensemble import VotingClassifier
clf = VotingClassifier([
                        ('clf1', RandomForestClassifier()),
                        ('clf2', LogisticRegression()),
                        ('clf3', KNeighborsClassifier()),
                        ('clf4', GaussianNB(var_smoothing=2)),
                        ('clf5', SVC()),
                        ('clf6', GaussianProcessClassifier()),
                       ])
clf.fit(X_tr.toarray(),y)

y_pred = clf.predict(x_test.toarray())
print('accuracy: ', accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy:  0.9140893470790378


We tried these models pretty naively here, they needs much more fine-tuning to find out the best possible hyper-parameters. Our goal was just get an estimate of how far different models can take us. It seems an accuracy around .91-.92 is the maxximum we can get right now and there isn't much difference when we ensemble models together in this case.  

So given that we have a moderate model, waht should be our recommendations to the library people? Let's take a look at feature-importances of our models:

For logistic models, it's just the coefficients of model parameters: 

In [114]:
log_model.coef_

array([[ 3.96558653e-01,  1.33895976e-02, -5.21663082e-01,
         2.02029591e-02,  1.81141653e-02,  9.65153896e-01,
         3.08446139e-02, -1.17191006e-01, -7.37073708e-02,
        -2.87734216e-01, -7.14276535e-02,  3.97821739e-01,
        -2.93685296e-02,  1.77142280e-01, -7.78874679e-02,
         4.51146022e-01,  3.83415458e-01, -4.82599173e-02,
        -3.98788492e-01, -6.28266208e-02, -3.13949173e-02,
        -4.74141016e-02, -5.64351884e-01, -7.16476348e-02,
         8.33854942e-02, -2.86474181e-01, -1.10890917e-01,
        -2.58013162e-01, -2.93371518e-02, -2.95373897e-01,
        -2.16682002e-01, -5.11843295e-02,  5.11729379e-02,
         4.47785442e-01, -2.05665039e-01, -8.93672263e-02,
        -3.38977312e-02, -1.97953478e-01,  3.46362654e-01,
         1.55601149e-01, -8.91074831e-02,  1.48213346e-01,
        -7.82305892e-02,  2.23010566e-01, -6.54578409e-02,
        -9.53447290e-02, -9.19066947e-02,  1.88743246e-01,
         3.09198600e-01, -5.74643901e-02,  2.28728265e-0

For our best model so far, GradientBoosting, we can get:

In [115]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf.fit(X_tr,y)

y_pred = clf.predict(x_test)
print('accuracy: ', accuracy_score(y_test, y_pred))
feature_import = clf.feature_importances_
print(feature_import)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy:  0.9209621993127147
[8.52764006e-02 7.54402996e-02 2.34004275e-01 1.17459171e-01
 1.82082157e-02 1.58463412e-01 5.26502496e-03 5.00750248e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.89738580e-03
 4.80399723e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 6.27740408e-04 2.73283754e-05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.23575575e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.0000000

This is very confusing since we have lots of features here:

In [116]:
len(feature_import)

787

This is due to lots of one-hot-encoding we applied to many features easpecially books' titles, topics, and authors. If the goal was just prediction, this procedure is okay; doesn't hurt if we have lots of features as long as our predictions get higher accuracy. But we lose interpretability by complicating our model and adding lots of features. Sicne here the goal is give some verbal advice to other human beings, let's try some simple models that can tell a story. 

Let's build a simple model with one variable: the libraries' ids: it should give us an estimate on how different branches vary from each other: 

In [124]:
X_merge_Y

,customer_id,book_id,library_id,lat_library,lng_library,delay,title,authors,publisher,publishedDate,categories,price,pages,birth_date,gender,education,occupation,lat_customer,lng_customer
0,8d3f63e1deed89d7ba1bf6a4eb101373,HUX-y4oXl04C,223-222@5xc-jxr-tgk,45.479005,-122.612107,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,5500,female,graduate,tech,45.486876,-122.637210
1,77a595f9fb214a8877c49f0c1e354f7e,HUX-y4oXl04C,23v-222@5xc-jv7-v4v,45.535336,-122.699145,1,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,11634,female,other,sales,45.485904,-122.704089
2,3fa0ad163470ae588dbff044328de641,HUX-y4oXl04C,222-222@5xc-jv5-nt9,45.562459,-122.671537,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,10019,female,NaN,sales,45.568862,-122.619258
3,230ea2e764085948ddf9d27371698b3a,HUX-y4oXl04C,zzw-222@5xc-knn-c5z,45.490566,-122.582281,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,5399,male,NaN,education_health,45.454092,-122.586715
4,8380f7f05a183e9b17f133d5e8a78707,HUX-y4oXl04C,226-222@5xc-jxj-7yv,45.515267,-122.622676,0,Indian Financial System 5E,['Khan'],Tata McGraw-Hill Education,64434.0,NaN,416.99,752,14886,female,graduate,tech,45.552633,-122.591968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,acd2c173eef280129c8782c164907fa9,N9V34rWEOGcC,222-222@5xc-kkw-bzf,45.516730,-122.538471,1,Financial Management,['Khan & Jain'],Tata McGraw-Hill Education,64768.0,['Corporations'],181.99,743,6416,male,graduate,education_health,45.503783,-122.479257
1447,3db968ed889159f4abb1b112a8221b4c,C4lNAAAAYAAJ,22d-222@5xc-kcy-8sq,45.467632,-122.652746,0,Mechanics of Engineering ...,['Irving Porter Church'],NaN,24837.0,['Engineering'],267.50,449,10226,NaN,highschool,business_finance,45.491433,-122.677191
1448,abcbf151a1dc88305fe989666d8ff52e,P1AoDwAAQBAJ,zzw-222@5xc-jv4-789,45.549385,-122.651389,0,Reality In Advertising,['Rosser Reeves'],Lulu.com,68460.0,['Business & Economics'],384.00,506,8103,male,graduate,admin_support,45.541832,-122.622660
1449,f106e9d626ec8ef9c1384efa24ef4de5,P1AoDwAAQBAJ,222-222@5xc-jvf-skf,45.590036,-122.751053,0,Reality In Advertising,['Rosser Reeves'],Lulu.com,68460.0,['Business & Economics'],384.00,506,21809,female,college,education_health,45.616595,-122.720332


In [137]:
onehot = OneHotEncoder()
y = df_train.delay.values.astype(int).reshape(-1,1)
X_train = onehot.fit_transform(df_train.library_id.values.reshape(-1,1))
X_test = onehot.transform(df_test.library_id.values.reshape(-1,1))
log_model = LogisticRegression()
log_model.fit(X_train, y)
y_pred = log_model.predict(X_test)
print('accuracy: ', accuracy_score(y_test, y_pred))

accuracy:  0.9140893470790378


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [150]:
tup = [(x,y) for x,y in zip(onehot.get_feature_names(), log_model.coef_[0])]
sorted(tup, key=lambda x:-x[1])

[('x0_228-222@5xc-jtz-hwk', 0.4920665347612279),
 ('x0_222-222@5xc-jxp-rp9', 0.3887072750719287),
 ('x0_23v-222@5xc-jv7-v4v', 0.35237246479948514),
 ('x0_zzw-224@5xc-jwv-2rk', 0.2527886242743661),
 ('x0_22d-222@5xc-kcy-8sq', 0.2251523566021076),
 ('x0_226-222@5xc-jxj-7yv', 0.19112129542482895),
 ('x0_zzw-222@5xc-knn-c5z', 0.18520907947432594),
 ('x0_225-222@5xc-jtz-hkf', 0.14788563961685186),
 ('x0_zzw-222@5xc-jv4-789', 0.13392313835458824),
 ('x0_223-222@5xc-jxr-tgk', 0.10663138143181093),
 ('x0_222-222@5xc-jv5-nt9', 0.05441085648282607),
 ('x0_222-222@5xc-jvf-skf', 0.044473642852124295),
 ('x0_zzw-223@5xc-jv7-ct9', 0.011731185080519752),
 ('x0_227-222@5xc-jww-btv', -0.18068453134359647),
 ('x0_22c-222@5xc-jwj-pvz', -0.321663422652173),
 ('x0_224-222@5xc-jw2-t9z', -0.48436776892528616),
 ('x0_222-222@5xc-kkw-bzf', -0.5352120559625377),
 ('x0_226-222@5xc-kc4-fpv', -1.0645390163976987)]

This is a significant result. It shows how libraries are ranked based on the overall delays in returns they got. Multnomah County Library at 205 NE Russell St has the largest delays. After that it is Hollywood Library at 4040 NE Tillamook St. The two lowest on delays are: Library Capitol Hill at 10723 SW capitol Hwy and MIDLAND at 805 SE 122nd Ave. One recommendation would be to do a close comparison between these two groups to find out why thay have this vast discrepancy. 

In [156]:
libraries[libraries.id=='228-222@5xc-jtz-hwk']['name'], libraries[libraries.id=='222-222@5xc-jxp-rp9']['name']

(11    Multnomah  County Library
 Name: name, dtype: object, 15     Multnomah  County  Library Hollywood Library
 Name: name, dtype: object)

In [161]:
libraries[libraries.id=='226-222@5xc-kc4-fpv'][['name', 'street_address']], libraries[libraries.id=='222-222@5xc-kkw-bzf'][['name', 'street_address']]

(                                        name        street_address
 0  Multnomah   County   Library Capitol Hill  10723 SW capitol Hwy,
                                 name    street_address
 10  MULTNOMAH County Library MIDLAND  805 SE 122nd Ave)

Let's build a similar model this time on just customers:

In [167]:
onehot = OneHotEncoder()
y = df_train.delay.values.astype(int).reshape(-1,1)
onehot.fit(X_merge_Y.customer_id.values.reshape(-1,1))
X_train = onehot.transform(df_train.customer_id.values.reshape(-1,1))
X_test = onehot.transform(df_test.customer_id.values.reshape(-1,1))
log_model = LogisticRegression()
log_model.fit(X_train, y)
y_pred = log_model.predict(X_test)
print('accuracy: ', accuracy_score(y_test, y_pred))

accuracy:  0.9140893470790378


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [168]:
tup = [(x,y) for x,y in zip(onehot.get_feature_names(), log_model.coef_[0])]
sorted(tup, key=lambda x:-x[1])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[('x0_006ec5b8f21fba3c5da132ee0f385a69', 0.8176325138513264),
 ('x0_00a03fa2d7f78b3a01a62beeb666cc34', 0.8176325138513264),
 ('x0_043f788a782b8933b5f208efefefb247', 0.8176325138513264),
 ('x0_04588e0e30daec741af385a0dab0a9bd', 0.8176325138513264),
 ('x0_0701c483239ce3146dc0103e67a1f589', 0.8176325138513264),
 ('x0_077af1a657c88e5ab8ac0cadfa64b3a0', 0.8176325138513264),
 ('x0_0a1ef2a67914bf221a7b6233e9f373cc', 0.8176325138513264),
 ('x0_0a5baae1b492a04e70678467b93ecef7', 0.8176325138513264),
 ('x0_0a7ffa18ed9f3eabe92329de44ee97cb', 0.8176325138513264),
 ('x0_0cccb436e836e9bb565127cdcf854efb', 0.8176325138513264),
 ('x0_0dd4585f087644eb3605d4cf6d1ed030', 0.8176325138513264),
 ('x0_162e9a61c7e7e1b4c0cdbe59c9aa13b6', 0.8176325138513264),
 ('x0_17a580b6122c944c77e084b399b1534a', 0.8176325138513264),
 ('x0_17d592b5583d6a8f7b008c93915798d6', 0.8176325138513264),
 ('x0_1a7d6c2a3bbbc30b42433b031a51eb2c', 0.8176325138513264),
 ('x0_1b2ed8978fcc7bd854c7b0c27ba6c2dd', 0.8176325138513264),
 ('x0_1d

It shows which customers have the longest history if delays.

Let's compare the effect of the distance between customers' homes to the library versus some other factors: books' price, number of pages, and publsihDate; and customers' birth_date and gender: 

In [194]:
X_merge_Y['distance'] = np.sqrt((X_merge_Y.lat_customer - X_merge_Y.lat_library)**2 + 
                                (X_merge_Y.lng_customer - X_merge_Y.lng_library)**2)

X_merge_Y['gen'] = (X_merge_Y['gender']=='male').astype(int).values
X_Y = X_merge_Y[['distance','birth_date','gen', 'pages', 'price', 'publishedDate','delay']]

df_train, df_test = train_test_split(X_Y, test_size=.2)

In [196]:
x = df_train[['distance','birth_date','gen', 'pages', 'price', 'publishedDate']]
y = df_train.delay.values.astype(int).reshape(-1,1)

impute = SimpleImputer()
scaler = StandardScaler()
x_tr = scaler.fit_transform(impute.fit_transform(x))

log_model = LogisticRegression()
log_model.fit(x_tr,y)

x_test = df_test[['distance','birth_date','gen', 'pages', 'price', 'publishedDate']]
y_test = df_test.delay.values.astype(int).reshape(-1,1)

y_pred = log_model.predict(scaler.transform(impute.transform(x_test)))
print('accuracy: ', accuracy_score(y_test, y_pred))

accuracy:  0.9518900343642611


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [197]:
log_model.coef_

array([[ 1.24192012, -0.28020759, -0.07094125,  1.10012226, -0.0965768 ,
        -0.12858976]])

In [206]:
tup = [(a,b) for a,b in zip(x_test.columns, log_model.coef_[0])]
sorted(tup, key=lambda x:-x[1])

[('distance', 1.2419201160808762),
 ('pages', 1.1001222591179558),
 ('gen', -0.07094124549523124),
 ('price', -0.09657679710918221),
 ('publishedDate', -0.1285897620638344),
 ('birth_date', -0.2802075879490793)]

If we got it right, it shows that customer's age and gender, and book's age and price doesn't affect the delays in return but interestingly, the distance from your home to the library does... And also the weight of the book! (As proxied by the total number of pages). So the heavier the book is and the farther you are from the library, it's much more likely that you procrastinate in returning the book.  